# EAS 4610
## Final Group Project
## Glacial Cycles (Pollard 1982)

### Dennis Frank
### Marcus Morris
### Nathan Warner

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags

# Introduction

# Equations

### Equation 3

In [20]:
r = 0.3
nu = 100 # km^2/yr
hprime0 = 0

t_f = 700000
x_f = 4884
dt = 50 #yr
dx = 55.5 #km
nt = int(t_f/dt) + 1
nx = int(x_f/dx) + 1
x = np.linspace(0, x_f, nx)
t = np.linspace(0, t_f, nt)

c_nu = (nu * dt) / (dx**2)

h_prime = np.zeros([nx, nt])
h_0 = np.zeros([nx]) ##initialize
h = np.zeros([nx, nt]) #placeholder

h_prime_data = np.array([c_nu*np.ones(nx), (1-2*c_nu)*np.ones(nx), c_nu*np.ones(nx)])
h_prime_diags = np.array([-1, 0, 1])
h_prime_M = spdiags(h_prime_data, h_prime_diags, nx, nx)

h_0_data = np.array([-c_nu*np.ones(nx), (2*c_nu)*np.ones(nx), -c_nu*np.ones(nx)])
h_0_diags = np.array([-1, 0, 1])
h_0_M = spdiags(h_0_data, h_0_diags, nx, nx)

h_data = np.array([r*c_nu*np.ones(nx), (-2*r*c_nu)*np.ones(nx), r*c_nu*np.ones(nx)])
h_diags = np.array([-1, 0, 1])
h_M = spdiags(h_data, h_diags, nx, nx)

## Boundary conditions

## Initial conditions

for k in np.arange(0, nt - 1):
    h_prime_new = np.matmul(h_prime_M, h_prime[:,k]) + np.matmul(h_0_M, h_0[:]) + np.matmul(h_M, h[:,k])
    h[:, k+1] = h_prime_new.transpose()

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

# Numerical Methods

# Results

# Figures

# Video Walkthrough

# References